### Imports
Import packages and load model parameters (uncertainties, levers, etc.)

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

import plotly.express as px
import plotly.graph_objects as go
from ema_workbench.analysis import (prim, dimensional_stacking)
from ema_workbench import ema_logging

In [ ]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
    Samplers,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation
from ema_workbench.analysis import parcoords


def sum_over(*args):
    return sum(args)

In [ ]:
# Enable logging
ema_logging.log_to_stderr(ema_logging.INFO)

### Policy formulations
Problem formulation 3 is selected, as we want to see the outcomes of each individual location to be able to zoom in on Zutphen.
In the following cells the wanted policies can be adjusted

In [ ]:
used_problem_formulation = 3

dike_model, planning_steps = get_model_for_problem_formulation(used_problem_formulation)

In [ ]:
# Create a function which sets everything to 0 by default
def get_0_dict():
    return {l.name: 0 for l in dike_model.levers}

# Creates a Policy object from a dict and a (optional) name
def create_policy(dict1, name=None):
    return Policy(f"Policy_{name}", **dict(get_0_dict(), **dict1))

In the following policies formulations:
- `switch` is to toggle room for river on and of
- The number after `DikeIncrease` is again the timestep, the value assigned to the thing the heightening in decimeters.
- Early Warning Systems can also be enabled, with `EWS_DaysToThreat`, which specifies the early warning time in days.

See the implementation in problem_formulation.py (starting from line 35) for more details.

In [1]:
pol_list = []

#Policies are created with a loop
for dm in [0, 10]:
    for location in [2]:
        for switch in [0,1]:
            for ews_days in [0, 4]:
                pol_list.append(create_policy({"A.3_DikeIncrease 0": dm, f"{location}_RfR 0": switch, "EWS_DaysToThreat": ews_days},
                                            name=f"Dike_{dm}dm_RfR_{location}{switch}_EWS_{ews_days}d"))


NameError: name 'create_policy' is not defined

### Run the model (or load the data)
In the next cell the model is ran (if `use_pickle1 = False`) and the new results data is saved, or, if `use_pickle1 = True`, the saved results data is loaded.

In [ ]:
# True, use results in pickle file; False, run MultiprocessingEvaluator
use_pickle1 = True

if use_pickle1:
    with open('data/formulation_results.pickle', 'rb') as filehandler:
        results = pickle.load(filehandler)

else:
    # pass the policies list to EMA workbench experiment runs
    n_scenarios = 15000
    with MultiprocessingEvaluator(dike_model, n_processes=10) as evaluator:
        results = evaluator.perform_experiments(n_scenarios, pol_list, uncertainty_sampling=Samplers.LHS)

    # Save results in Pickle file
    with open('data/policyresults.pickle', 'wb') as filehandler:
        pickle.dump(results, filehandler)

### Clean up the results
The data is cleaned, policies renamed, and new columns created

In [ ]:
#dictionary to rename the policies
#so we don't have to run everything again
policy_dict = {
    'Policy_Dike_0dm_RfR_20_EWS_0d': 'Policy 0', 
    'Policy_Dike_0dm_RfR_20_EWS_4d': 'Policy 1: EWS',
    'Policy_Dike_0dm_RfR_21_EWS_0d': 'Policy 2: RfR', 
    'Policy_Dike_0dm_RfR_21_EWS_4d': 'Policy 3: RfR+EWS',
    'Policy_Dike_10dm_RfR_20_EWS_0d': 'Policy 4: Dike',
    'Policy_Dike_10dm_RfR_20_EWS_4d': 'Policy 5: Dike+EWS',
    'Policy_Dike_10dm_RfR_21_EWS_0d': 'Policy 6: Dike+RfR', 
    'Policy_Dike_10dm_RfR_21_EWS_4d': 'Policy 7: Dike+RfR+EWS'
    }


In [ ]:
# Create a dataframe from outcomes, and add the policy column to it
exp, out = results
df = pd.DataFrame(out)

#calculates the total costs for all locations
a = df.columns[df.columns.str.contains('Costs')]
df['Total Costs'] = df[a].sum(axis=1)
#calculates deaths for all locations
a = df.columns[df.columns.str.contains('Deaths')]
df['Total Deaths'] = df[a].sum(axis=1)

#renames the policies
exp['policy'] = exp.policy.map(policy_dict)

df["policy"] = pd.DataFrame(exp)["policy"]
df

In [ ]:
with open('data/policyresults.pickle', 'wb') as filehandler:
    pickle.dump(df, filehandler)

### Visualize the results
In this section the data is visualized in several plots

In [ ]:
#creates pairplot and saves it in a png
#plotting it in the notebook takes a while
data=df.drop(['policy'], axis=1)
sns_plot = sns.pairplot(df, hue='policy',  vars=data.columns )
sns_plot.savefig('pairplot.png')
#plt.show()

In [ ]:
#to create a boxplot the df first has to be "melted"
df_melt = pd.melt(df, id_vars='policy')

#15 subplots are created (even though only 14 were necessary)
#it looks better in the report with three columns
f, axes = plt.subplots(5, 3, sharex=True, figsize=(30, 50))

#unique outcome list with a count to loop through and create a boxplot for each outcome
unq_pol = df_melt.variable.unique()
count = 0

#creates boxplot
axes = axes.flatten()
for axs in axes:
        if count<14:
                print(axs)
                sns.boxplot(x="policy", y="value", data=df_melt.loc[df_melt.variable ==unq_pol[count]], ax=axs)
                axs.set_xticklabels(axs.get_xticklabels(),rotation=90)
                axs.set_title(unq_pol[count])
                count+=1
        else:
                pass
f.savefig('boxplot.png')
plt.show()

In [ ]:
#rename to make the plots look nicer
df.rename({
       'A.1 Total Costs': '1.C', 'A.1_Expected Number of Deaths': '1.D',
       'A.2 Total Costs': '2.C', 'A.2_Expected Number of Deaths': '2.D' , 'A.3 Total Costs': '3.C',
       'A.3_Expected Number of Deaths': '3.D', 'A.4 Total Costs': '4.C',
       'A.4_Expected Number of Deaths': '4.D', 'A.5 Total Costs': '5.C',
       'A.5_Expected Number of Deaths': '5.D',  'RfR Total Costs': 'RfR', 'Expected Evacuation Costs':'Evac Costs'
       },
       axis=1, inplace=True)

In [ ]:
pol_list = df['policy'].unique()

#policies have to be renamed to numbers as plotly parallel coordinates are not able to take non numbers as color
count=0
for i in pol_list:
    
    df.loc[df.policy== i, 'policy'] = count
    count+=1

#get the limits of the variables    
limits = parcoords.get_limits(df.iloc[:,:-1])

#create a list for all dimensions to be plotted
dimensionlist = []

#dimensionlist is filled with names and limits 
for column in limits:
    lower=0
    upper=limits[column].iloc[1]
    if upper>0:
        dimensionlist.append(dict(range = [lower,upper],
                label = column, values = df[column].values, tickformat = "~g"))
    else:
        dimensionlist.append(dict(range = [lower,0.1],
                label = column, values = df[column].values, tickformat = "~g"))

#plots parallel plot
fig = go.Figure(data=
    go.Parcoords(line = dict(color = df['policy'], showscale=True),
                   dimensions= dimensionlist
                             ))
plt.savefig('explorationparallelplot.png')
fig.show()

### Prim, scenario discovery
Thresholds are chosen for prim to select a smaller set of scenarios that will be analyzed. This set of scenarios consists of cases with expected deaths greater than zero and cases with total costs > 250 million euros

In [ ]:
#select prefered policies as mentioned in report
exp_select = exp.loc[exp.policy.isin(['Policy 5: Dike+EWS', 'Policy 3: RfR+EWS','Policy 7: Dike+RfR+EWS'])]
exp_select.reset_index(inplace=True)
out_select = df.loc[exp.policy.isin(['Policy 5: Dike+EWS', 'Policy 3: RfR+EWS','Policy 7: Dike+RfR+EWS'])]
out_select.reset_index(inplace=True)

#clean up experiments
x= exp_select[[
       'A.0_ID flood wave shape', 'A.1_Bmax', 'A.1_Brate', 'A.1_pfail',
       'A.2_Bmax', 'A.2_Brate', 'A.2_pfail', 'A.3_Bmax', 'A.3_Brate',
       'A.3_pfail', 'A.4_Bmax', 'A.4_Brate', 'A.4_pfail', 'A.5_Bmax',
       'A.5_Brate', 'A.5_pfail', 'discount rate 0', 'discount rate 1', 'policy'
       ]]

#create conditions
y1=out_select['A.3_Expected Number of Deaths'] >0
y2=out_select['A.3 Total Costs'] > 250e6
y= np.logical_or(y1,y2)



In [ ]:
# put y1 or y2 if you want them seperately: prim_alg = prim.Prim(x, y, threshold=0.7)
prim_alg = prim.Prim(x, y, threshold=0.7)
box1 = prim_alg.find_box()

In [ ]:
box1.show_tradeoff()
plt.show()

In [ ]:
box1.inspect_tradeoff()

In [ ]:
box1.show_pairs_scatter()
plt.show()

In [ ]:
box1.inspect(15, style='graph')
plt.show()

In [ ]:
dimensional_stacking.create_pivot_plot(x, y)
plt.show()